<a href="https://colab.research.google.com/github/jacksonchen1998/NTUT-Machine-Leraning-Course/blob/main/Homework_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

UC Irvine has a large repository for various kinds of data. 

In this problem, you are asked to use the iris dataset (https://archive.ics.uci.edu/ml/datasets/Iris) to perform the experiments. 

To begin one trial, randomly draw 70% of the instances for training and the rest for testing. 

Note: you can directly import iris dataset by using sklearn without downloading from the UC Irvine repository. 

Using GMM with 2 mixtures instead. 

The GMM tools are supported in sklearn. 

Remember to use one model per class. Of the k-NN, Naïve Bayes, and GMM classifiers, which one has the best accuracy?

In [9]:
import matplotlib as mpl
import matplotlib.pyplot as plt

import numpy as np

from sklearn import datasets # import dataset
from sklearn.mixture import GaussianMixture # import GMM
from sklearn.model_selection import train_test_split # import split function 


In [21]:
colors = ["orange", "blue"]

def make_ellipses(gmm, ax):
    for n, color in enumerate(colors):
        if gmm.covariance_type == "full":
            covariances = gmm.covariances_[n][:2, :2]
        elif gmm.covariance_type == "tied":
            covariances = gmm.covariances_[:2, :2]
        elif gmm.covariance_type == "diag":
            covariances = np.diag(gmm.covariances_[n][:2])
        elif gmm.covariance_type == "spherical":
            covariances = np.eye(gmm.means_.shape[1]) * gmm.covariances_[n]
        v, w = np.linalg.eigh(covariances)
        u = w[0] / np.linalg.norm(w[0])
        angle = np.arctan2(u[1], u[0])
        angle = 180 * angle / np.pi  # convert to degrees
        v = 2.0 * np.sqrt(2.0) * np.sqrt(v)
        ell = mpl.patches.Ellipse(
            gmm.means_[n, :2], v[0], v[1], 180 + angle, color=color
        )
        ell.set_clip_box(ax.bbox)
        ell.set_alpha(0.5)
        ax.add_artist(ell)
        ax.set_aspect("equal", "datalim")

In [ ]:
iris = datasets.load_iris()

# Break up the dataset into non-overlapping training (70%) and testing
# (30%) sets.

X_train, X_test, y_train, y_test = train_test_split(iris.data,iris.target,test_size=0.3)

n_classes = len(np.unique(y_train))

# Try GMMs using different types of covariances.
estimators = {
    cov_type: GaussianMixture(
        n_components=n_classes, covariance_type=cov_type, max_iter=20, random_state=0
    )
    for cov_type in ["spherical", "diag", "tied", "full"]
}

n_estimators = len(estimators)

plt.figure(figsize=(3 * n_estimators // 2, 6))
plt.subplots_adjust(
    bottom=0.01, top=0.95, hspace=0.15, wspace=0.05, left=0.01, right=0.99
)


for index, (name, estimator) in enumerate(estimators.items()):
    # Since we have class labels for the training data, we can
    # initialize the GMM parameters in a supervised manner.
    estimator.means_init = np.array(
        [X_train[y_train == i].mean(axis=0) for i in range(n_classes)]
    )

    # Train the other parameters using the EM algorithm.
    estimator.fit(X_train)

    h = plt.subplot(2, n_estimators // 2, index + 1)
    make_ellipses(estimator, h)

    for n, color in enumerate(colors):
        data = iris.data[iris.target == n]
        plt.scatter(
            data[:, 0], data[:, 1], s=0.8, color=color, label=iris.target_names[n]
        )
    # Plot the test data with crosses
    for n, color in enumerate(colors):
        data = X_test[y_test == n]
        plt.scatter(data[:, 0], data[:, 1], marker="x", color=color)

    y_train_pred = estimator.predict(X_train)
    train_accuracy = np.mean(y_train_pred.ravel() == y_train.ravel()) * 100
    plt.text(0.05, 0.9, "Train accuracy: %.1f" % train_accuracy, transform=h.transAxes)

    y_test_pred = estimator.predict(X_test)
    test_accuracy = np.mean(y_test_pred.ravel() == y_test.ravel()) * 100
    plt.text(0.05, 0.8, "Test accuracy: %.1f" % test_accuracy, transform=h.transAxes)

    plt.xticks(())
    plt.yticks(())
    plt.title(name)

plt.legend(scatterpoints=1, loc="lower right", prop=dict(size=12))
plt.show()

In this problem, you are asked to perform the wrapper-type feature selection using the Naïve Bayes classifier for cancer dataset 

(Breast Cancer Wisconsin (Original) Data Set, directly from the sklearn or downloading 

from https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Original%29). 

To simplify the problem, we just want to keep 5 attributes out of 9 (hint: one attribute is useless. Which one is it?) 

To begin one experiment, randomly draw 60 % of the instances from each class for training, 

and 20% from each class for finding the best 5 attributes. 

Once the feature selection is complete, use the rest 20% for testing to obtain the accuracy. 

Repeat the selection 10 times to get the average accuracy. 

You need to deal with missing attributes. 

Compare the obtained accuracy with the same type of model trained with the full set of 9 features.

In [ ]:
import pandas as pd

url = "https://drive.google.com/file/d/1foB8RSromI9rH_5nRaHubkr12zkElmmL/view?usp=sharing"
url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url, header=None) #file contains no header info
print(f"Read in {len(df)} rows")
df.head()

In [ ]:
df.replace("?", 10000, inplace=True) #10,000 is way beyond the range of columns provided so acts as an outlier
df.drop([0], 1, inplace=True)

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import feature_selection as fs
from sklearn.metrics import accuracy_score

X = np.array(df.drop([10], 1)) #last column contains label, so ignore it when creating X
y = np.array(df[10]) #last column contains label which is our y

X_new = fs.SelectKBest(fs.chi2,k=5).fit_transform(X, y) # Using feature selection

sum = 0

for _ in range(10):
  X_train, X_val, y_train, y_val = train_test_split(X_new, y, test_size=0.4, random_state=43)
  X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=43)

  gnb_model = GaussianNB()
  y_result = gnb_model.fit(X_train, y_train).predict(X_test)
  sum += accuracy_score(y_test, y_result)

print(f"Accuracy of Gaussian Naive Bayes Classifier is:{sum / 10}")

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import feature_selection as fs
from sklearn.naive_bayes import GaussianNB

X = np.array(df.drop([10], 1)) #last column contains label, so ignore it when creating X
y = np.array(df[10]) #last column contains label which is our y

sum = 0 # Original one using Naive Bayes Classify

for _ in range(10):
  X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=43)
  X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=43)

  gnb_model = GaussianNB()
  y_result = gnb_model.fit(X_train, y_train).predict(X_test)
  sum += accuracy_score(y_test, y_result)

print(f"Accuracy of Gaussian Naive Bayes Classifier is:{sum / 10}")